In [2]:

import os 
from Emotions_Detection.Models.features import lpq, lpq_plus, LPQ, LPQPlus, PHOG_Algorithm

ModuleNotFoundError: No module named 'Emotions_Detection'

In [ ]:
# Constants and Vaiables
IMG_TRAIN_PATH = "../Assets/data/train"
IMG_TEST_PATH = "../Assets/data/test"
HAPPY_PATH = "../Assets/data/train/happy/"

train_folders = os.listdir(IMG_TRAIN_PATH)  # Emotions Folders
test_folders = os.listdir(IMG_TEST_PATH)  # Emotions Folders

# Intialize dash board for DASK
# client = dask.distributed.Client()  # Create a local cluster  
cluster = dask.distributed.LocalCluster(n_workers=12, threads_per_worker=2)
client = dask.distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 15.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59841,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 15.91 GiB
Comm: tcp://127.0.0.1:59936,Total threads: 2
Dashboard: http://127.0.0.1:59939/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:59847,


## Different Methods for Reading the Images:
**1- Using Separete Delayed Functions.**  
_Quiet Fast ~ 1.5 mins_

In [ ]:
# Reading Training Images ~ 1.6 mins
x_img = []
y_train = []
tasks = []
# Create the reading Tasks and add them to a list -> tasks
for folder in train_folders:
    folder_path = os.path.join(IMG_TRAIN_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    tasks.append(imgs)
    # Set Labels While Initializing
    labels = [folder for _ in range(len(imgs))]
    y_train.extend(labels) 
    
# Distribute Tasks to Cores and Threads 
print("Implementing Tasks...")
for task in tasks:
    x_img.extend(task.compute())
    print("Done...")

In [ ]:
# Reading Test Images ~ 30 sec
x_test_img = []
y_test = []
tasks = []
# Create the reading Tasks and add them to a list -> tasks


# Distribute Tasks to Cores and Threads 
print('implementing Tasks')
for task in tasks:
    x_test_img.extend(task.compute())
    print("Done...")

In [ ]:
# Verifying Results
len(y_train), len(x_img), len(x_test_img), len(y_test)

(28709, 28709, 7178, 7178)

**2- Using list of delayed Functions.**  
_Looks Too Slow ~ 5 mins_

In [ ]:
# Reading Train Dataset
x_img = []
y_img = []
for folder in train_folders:
    folder_path = os.path.join(IMG_TRAIN_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    x_img.extend(imgs)
    labels = [folder for _ in range(len(imgs))]
    y_img.extend(labels) 

print("Implementing Tasks")
x_img = dask.compute(x_img)[0]
print('Done...')

In [ ]:
# Reading Test Dataset
x_test_img = []
y_test = []
for folder in test_folders:
    folder_path = os.path.join(IMG_TEST_PATH, folder)
    pattern = os.path.join(folder_path, 'im*.png')
    imgs = dask_image.imread.imread(pattern)
    x_test_img.extend(imgs)
    labels = [folder for _ in range(len(imgs))]
    y_test.extend(labels) 

print("implementing Reads")
x_test_img = dask.compute(x_test_img)[0]
print("Done...")

#### **Applying Feature Extraction for all the images**

#####  1- LPQ and LPQ_PLUS.   

In [ ]:
# Training and Test Images LPQ Feature Extraction
lpq_train = []
for img in x_img:
    lpq_desc = dask.delayed(lpq)(img)
    lpq_train.append(lpq_desc)

print('Starting LPQ Train Set...')
lpq_train = dask.compute(lpq_train)[0]
print('Done LPQ Train Set...')

# Testing Images
lpq_test = []
for img in x_test_img:
    lpq_desc = dask.delayed(lpq)(img)
    lpq_test.append(lpq_desc)

print('Starting LPQ Test Set...')
lpq_test = dask.compute(lpq_test)[0]
print('Done LPQ Test Set...')

### PHOG Feature Extraction -> Very Slow!!!

In [ ]:
# Training and Test Images PHOG Feature Extraction
# Training Images
phog_train = []
for img in x_img:
    phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
    phog_train.append(phog_desc)

print('Starting...')
phog_train = dask.compute(phog_train)[0]
print('Done...')

# Test Images
phog_test = []
for img in x_test_img:
    phog_desc = dask.delayed(PHOG_Algorithm)(img, 8, 3)
    phog_test.append(phog_desc)

print('Starting...')
phog_test = dask.compute(phog_test)[0]
print('Done...')

In [ ]:
phog_train[0].shape, lpq_train[0].shape

((680,), (255,))

In [ ]:
training_data = np.concatenate((lpq_train, phog_train), axis=1)
testing_data = np.concatenate((lpq_test, phog_test), axis=1)
training_data.shape, testing_data.shape, (255+680)

((28709, 935), (7178, 935), 935)